In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import KFold
from numpy import mean, argmax

import loadDataset
import models
import augment

In [ ]:
# No longer shrinking images. This will crash your computer.
#X, Y = loadDataset.loadDataSet('./dataset/imgs/rock_frames', './dataset/imgs/paper_frames',
#                               './dataset/imgs/scissor_frames', './dataset/csvs/rock.csv',
#                               './dataset/csvs/paper.csv', './dataset/csvs/scissor.csv')
#X = X.astype(np.float32) / 255

#X1, Y1 = X, (Y > 0).astype(int)  # 0: None/NA, 1: Rock/Paper/Scissors
#X2, Y2 = X[Y > 0], Y[Y > 0] - 1  # Y is 0-based containing only Rock, Paper, or Scissors.

In [2]:
def _csv_to_list(path):
    with open(path) as f:
        return f.read().splitlines()


def _load_images(paths: str or list or tuple) -> np.array:
    if isinstance(paths, str):
        paths = [paths]
    imgs = np.array([cv2.cvtColor(cv2.imread(path, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB) for path in paths]).astype(float) / 255
    return imgs if imgs.shape[0] > 1 else imgs.reshape(imgs.shape[1:])

In [3]:
def video_batch_generator_augment(img_paths, csv_paths, session, batch_size=64, vid_length=45, shuffle=False):
    if not isinstance(img_paths, (tuple, list)):
        img_paths = [img_paths]
    if not isinstance(csv_paths, (tuple, list)):
        csv_paths = [csv_paths]
        
    csv_files = [[int(x) for x in _csv_to_list(path)] for path in csv_paths]
    img_files = [[path + "/" + str(i) + ".jpeg" for i in range(1, len(csv_files[j]) + 1)] for j, path in enumerate(img_paths)]
    csv_files = np.concatenate(np.array(csv_files))
    img_files = [item for sublist in img_files for item in sublist]  # Flatten to 1D
    
    img_files, csv_files = [f for i, f in enumerate(img_files) if csv_files[i] > 0], csv_files[csv_files > 0] - 1  # Y is 0-based containing only Rock, Paper, or Scissors.
    total_length = len(csv_files) - vid_length  # Number of video starting points

    image_shape = _load_images(img_files[0]).shape
    oper = augment._augment_video_oper(img_size=image_shape, vid_length=vid_length)
    order = np.random.permutation(total_length) if shuffle is True else np.arange(total_length)
    
    for i in range(int(total_length / batch_size)):
        start_indices = order[(i)*batch_size:(i+1)*batch_size]
        print(start_indices)
        x_batch = [_load_images(img_files[(start):(start+vid_length)]) for start in start_indices]
        y_batch = [csv_files[(start):(start+vid_length)] for start in start_indices]

        for j, x_vid in enumerate(x_batch):
            x_batch[j] = augment._augment_video(x_vid, session, oper)

        yield np.stack(x_batch), np.stack(y_batch)[:, -1]
        
        
def image_batch_generator_augment(img_paths, csv_paths, session, batch_size=64, shuffle=False):
    if not isinstance(img_paths, (tuple, list)):
        img_paths = [img_paths]
    if not isinstance(csv_paths, (tuple, list)):
        csv_paths = [csv_paths]
        
    csv_files = [[int(x) for x in _csv_to_list(path)] for path in csv_paths]
    img_files = [[path + "/" + str(i) + ".jpeg" for i in range(1, len(csv_files[j]) + 1)] for j, path in enumerate(img_paths)]
    csv_files = np.concatenate(np.array(csv_files))
    img_files = [item for sublist in img_files for item in sublist]  # Flatten to 1D
    
    img_files, csv_files = img_files, (csv_files > 0).astype(int)  # 0: None/NA, 1: Rock/Paper/Scissors
    total_length = len(csv_files)  # Number of images

    image_shape = _load_images(img_files[0]).shape
    oper = augment._augment_video_oper(img_size=image_shape, vid_length=vid_length)
    order = np.random.permutation(total_length) if shuffle is True else np.arange(total_length)
    
    for i in range(int(total_length / batch_size)):
        indices = order[(i)*batch_size:(i+1)*batch_size]
        x_batch = [_load_images(img_files[j]) for j in indices]
        y_batch = csv_files[indices]

        for j, x_img in enumerate(x_batch):
            x_batch[j] = _augment_video(x_img[None, :], session, oper)

        yield np.stack(x_batch), y_batch

### Fit Models

In [4]:
def fit1(session, model, img_paths, csv_paths, batch_size=64, epochs=5, shuffle=False, verbose=True, print_interval=100):
    [predict_op, loss_op, accuracy_op, train_op], (X, Y, training) = model
    
    session.run(tf.global_variables_initializer())
    session.run(tf.local_variables_initializer())
    
    train_loss, train_accuracy = [], []
    valid_loss, valid_accuracy = [], []
    
    # Training
    for e in range(epochs):
        sum_loss, sum_accuracy, num_batches = 0, 0, 0
        for batch_i, (batch_x, batch_y) in enumerate(video_batch_generator_augment(img_paths, csv_paths, session, batch_size=batch_size, vid_length=45, shuffle=shuffle)):
            loss, accuracy, _ = session.run([loss_op, accuracy_op, train_op],
                                            feed_dict={X: batch_x, Y: batch_y, training: True})

            if verbose and batch_i % print_interval == 0:
                print("Train Batch {}: Loss = {}, Accuracy = {}".format(batch_i, loss, accuracy))
            sum_loss += loss
            sum_accuracy += accuracy
            num_batches += 1

        train_loss.append(sum_loss / num_batches)
        train_accuracy.append(sum_accuracy / num_batches)
        if verbose:
            print("Epoch {}: Average Train Loss = {}, Average Train Accuracy = {}\n"
                  .format(e + 1, train_loss[e], train_accuracy[e]))
    
    plt.figure()
    plt.title("Training Loss per Epoch")
    plt.plot(np.arange(epochs), np.array(train_loss), label="Training")
    plt.xlabel("Epoch")
    plt.xticks(np.arange(epochs), np.arange(epochs))
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
    
def fit2(session, model, img_paths, csv_paths, batch_size=64, epochs=5, shuffle=False, verbose=True, print_interval=100):
    [predict_op, loss_op, accuracy_op, train_op], (X, Y, training) = model
    
    session.run(tf.global_variables_initializer())
    session.run(tf.local_variables_initializer())
    
    train_loss, train_accuracy = [], []
    valid_loss, valid_accuracy = [], []
    
    # Training
    for e in range(epochs):
        sum_loss, sum_accuracy, num_batches = 0, 0, 0
        for batch_i, (batch_x, batch_y) in enumerate(image_batch_generator_augment(img_paths, csv_paths, session, batch_size=batch_size, shuffle=shuffle)):
            loss, accuracy, _ = session.run([loss_op, accuracy_op, train_op],
                                            feed_dict={X: batch_x, Y: batch_y, training: True})

            if verbose and batch_i % print_interval == 0:
                print("Train Batch {}: Loss = {}, Accuracy = {}".format(batch_i, loss, accuracy))
            sum_loss += loss
            sum_accuracy += accuracy
            num_batches += 1

        train_loss.append(sum_loss / num_batches)
        train_accuracy.append(sum_accuracy / num_batches)
        if verbose:
            print("Epoch {}: Average Train Loss = {}, Average Train Accuracy = {}\n"
                  .format(e + 1, train_loss[e], train_accuracy[e]))
    
    plt.figure()
    plt.title("Training Loss per Epoch")
    plt.plot(np.arange(epochs), np.array(train_loss), label="Training")
    plt.xlabel("Epoch")
    plt.xticks(np.arange(epochs), np.arange(epochs))
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

In [ ]:
tf.reset_default_graph()
with tf.Session() as session:
    image_size = [64, 64, 3]
    img_paths = ["./dataset/imgs/rock_frames", "./dataset/imgs/paper_frames", "./dataset/imgs/scissor_frames"]
    csv_paths = ["./dataset/csvs/rock.csv", "./dataset/csvs/paper.csv", "./dataset/csvs/scissor.csv"]
    model1 = models.model1(image_size, image_history_length=45)
    fit1(session, model1, img_paths, csv_paths, epochs=5, shuffle=True, verbose=True, print_interval=10)
    model2 = models.model2(image_size)
    fit2(session, model2, img_paths, csv_paths, epochs=2, shuffle=True, verbose=True, print_interval=10)
    
    # Save Model
    # https://www.tensorflow.org/programmers_guide/saved_model
    save_path = os.path.join(os.getcwd(), "savedmodels\\both\\models.ckpt")
    saver = tf.train.Saver()
    saver.save(session, save_path)
    print("Session Saved.")

### Cross-Validate Models

In [ ]:
# Custom Cross-Validation that uses Custom Batch Generator
def cross_validation1(session, model, x, y, batch_size=64, epochs=5, K=5, shuffle=False, verbose=True, print_interval=100):
    # https://stackoverflow.com/questions/39748660/how-to-perform-k-fold-cross-validation-with-tensorflow
    [predict_op, loss_op, accuracy_op, train_op], (X, Y, training) = model

    # K-Fold Loop
    train_loss, train_accuracy = [], []
    valid_loss, valid_accuracy = [], []
    k = 0
    for train_i, valid_i in KFold(n_splits=K).split(x):
        train_loss.append([])
        train_accuracy.append([])

        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())

        num_train_batches = int(x[train_i].shape[0] / batch_size)
        num_valid_batches = int(x[valid_i].shape[0] / batch_size)

        # Training
        for e in range(epochs):
            sum_loss, sum_accuracy = 0, 0
            for batch_i, (batch_x, batch_y) in enumerate(_batches_video([x[train_i], y[train_i]], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
                batch_y = batch_y[:, -1]
                loss, accuracy, _ = session.run([loss_op, accuracy_op, train_op],
                                                feed_dict={X: batch_x, Y: batch_y, training: True})

                if verbose and batch_i % print_interval == 0:
                    print("Train Batch {}: Loss = {}, Accuracy = {}".format(batch_i, loss, accuracy))
                sum_loss += loss
                sum_accuracy += accuracy

            train_loss[k].append(sum_loss / num_train_batches)
            train_accuracy[k].append(sum_accuracy / num_train_batches)
            if verbose:
                print("Epoch {}: Average Train Loss = {}, Average Train Accuracy = {}\n"
                      .format(e + 1, train_loss[k][e], train_accuracy[k][e]))

        # Validation
        sum_loss, sum_accuracy = 0, 0
        for batch_i, (batch_x, batch_y) in enumerate(_batches_video([x[valid_i], y[valid_i]], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
            batch_y = batch_y[:, -1]
            loss, accuracy = session.run([loss_op, accuracy_op],
                                         feed_dict={X: batch_x, Y: batch_y, training: False})

            if verbose and batch_i % print_interval == 0:
                print("Valid Batch {}: Loss = {}, Accuracy = {}".format(batch_i, loss, accuracy))
            sum_loss += loss
            sum_accuracy += accuracy

        valid_loss.append(sum_loss / num_valid_batches)
        valid_accuracy.append(sum_accuracy / num_valid_batches)
        if verbose:
            print("Fold {}: Validation Loss = {}, Validation Accuracy = {}\n"
                  .format(k + 1, valid_loss[k], valid_accuracy[k]))

        k += 1

    # Results
    print("Average Valid Loss = {}, Average Valid Accuracy = {}".format(mean(valid_loss), mean(valid_accuracy)))

    plt.figure()
    plt.title("Training Loss per Epoch")
    plt.plot(np.arange(epochs), np.array(train_loss).T)
    plt.xlabel("Epoch")
    plt.xticks(np.arange(epochs), np.arange(epochs))
    plt.ylabel("Loss")
    plt.legend(["Fold %d" % i for i in range(1, K+1)])
    plt.show()
    
def cross_validation2(session, model, x, y, batch_size=64, epochs=5, K=5, shuffle=False, verbose=True, print_interval=100):
    # https://stackoverflow.com/questions/39748660/how-to-perform-k-fold-cross-validation-with-tensorflow
    [predict_op, loss_op, accuracy_op, train_op], (X, Y, training) = model

    # K-Fold Loop
    train_loss, train_accuracy = [], []
    valid_loss, valid_accuracy = [], []
    k = 0
    for train_i, valid_i in KFold(n_splits=K).split(x):
        train_loss.append([])
        train_accuracy.append([])

        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())

        num_train_batches = int(x[train_i].shape[0] / batch_size)
        num_valid_batches = int(x[valid_i].shape[0] / batch_size)

        # Training
        for e in range(epochs):
            sum_loss, sum_accuracy = 0, 0
            for batch_i, (batch_x, batch_y) in enumerate(_batches([x[train_i], y[train_i]], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
                loss, accuracy, _ = session.run([loss_op, accuracy_op, train_op],
                                                feed_dict={X: batch_x, Y: batch_y, training: True})

                if verbose and batch_i % print_interval == 0:
                    print("Train Batch {}: Loss = {}, Accuracy = {}".format(batch_i + 1, loss, accuracy))
                sum_loss += loss
                sum_accuracy += accuracy

            train_loss[k].append(sum_loss / num_train_batches)
            train_accuracy[k].append(sum_accuracy / num_train_batches)
            if verbose:
                print("Epoch {}: Average Train Loss = {}, Average Train Accuracy = {}\n"
                      .format(e + 1, train_loss[k][e], train_accuracy[k][e]))

        # Validation
        sum_loss, sum_accuracy = 0, 0
        for batch_i, (batch_x, batch_y) in enumerate(_batches([x[valid_i], y[valid_i]], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
            loss, accuracy = session.run([loss_op, accuracy_op],
                                         feed_dict={X: batch_x, Y: batch_y, training: False})

            if verbose and batch_i % print_interval == 0:
                print("Valid Batch {}: Loss = {}, Accuracy = {}".format(batch_i + 1, loss, accuracy))
            sum_loss += loss
            sum_accuracy += accuracy

        valid_loss.append(sum_loss / num_valid_batches)
        valid_accuracy.append(sum_accuracy / num_valid_batches)
        if verbose:
            print("Fold {}: Validation Loss = {}, Validation Accuracy = {}\n"
                  .format(k + 1, valid_loss[k], valid_accuracy[k]))

        k += 1

    # Results
    print("Average Valid Loss = {}, Average Valid Accuracy = {}".format(mean(valid_loss), mean(valid_accuracy)))

    plt.figure()
    plt.title("Training Loss per Epoch")
    plt.plot(np.arange(epochs), np.array(train_loss).T)
    plt.xlabel("Epoch")
    plt.xticks(np.arange(epochs), np.arange(epochs))
    plt.ylabel("Loss")
    plt.legend(["Fold %d" % i for i in range(1, K+1)])
    plt.show()

In [ ]:
tf.reset_default_graph()
with tf.Session() as session:
    image_shape = X1.shape[1:]
    model1 = models.model1(image_shape, 45)
    cross_validation1(session, model1, X1, Y1, epochs=5, shuffle=True, print_interval=10)

In [ ]:
tf.reset_default_graph()
with tf.Session() as session:
    image_shape = X2.shape[1:]
    mode2 = models.model2(image_shape)
    cross_validation2(session, model2, X2, Y2, epochs=5, shuffle=True, print_interval=10)